Importing Libraries

In [1]:
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

Using URLLIB to get the page

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page, 'html5lib')

Using BeautifulSoup to get table on the page

In [3]:
all_table = soup.find_all('table')
table = soup.find('table', class_='wikitable sortable')
#print(table)

In [4]:
A=[]
B=[]
C=[]

for row in table.findAll("tr"): #para tudo que estiver em <tr>
    cells = row.findAll('td') #variável para encontrar <td>
    if len(cells)==3: #número de colunas
        A.append(cells[0].find(text=True)) #iterando sobre cada linha
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [5]:
df = pd.DataFrame(index=A, columns=['Postal'])

df['Postal']=A
df['Borough']=B
df['Neighboorhood']=C

df.head()

,Postal,Borough,Neighboorhood
M1A,M1A,Not assigned,Not assigned
M2A,M2A,Not assigned,Not assigned
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,Harbourfront


In [6]:
import numpy as np
df.replace('Not assigned', np.nan, inplace = True)
df.head(5)

,Postal,Borough,Neighboorhood
M1A,M1A,NaN,Not assigned
M2A,M2A,NaN,Not assigned
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,Harbourfront


In [7]:
df.dropna(subset=["Borough"], axis=0, inplace = True)
df.reset_index(drop = True, inplace = True)
df.head()

,Postal,Borough,Neighboorhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
df2 = df.groupby(['Postal','Borough'])['Neighboorhood'].apply(', '.join).reset_index()

#df2 = df.groupby(['Postal', 'Borough']).agg(lambda col: ','.join(col))
df2.head()

,Postal,Borough,Neighboorhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
path = 'http://cocl.us/Geospatial_data'
df3 = pd.read_csv(path)
df3.rename(columns={'Postal Code':'Postal'}, inplace=True)
df3.head()

,Postal,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_junto = pd.merge(df2, df3, on=['Postal'], how='inner')
df_junto.head()


,Postal,Borough,Neighboorhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
